In [32]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# CSV 파일 읽기
df = pd.read_csv('dc_(ID,Artist).csv')

# Artist 열의 첫 번째 데이터 (가수명) 가져오기
first_artist = df['Artist'].iloc[0]  # 첫 번째 아티스트 선택

# 수집할 날짜 범위 설정
start_date = datetime(2023, 9, 24)
end_date = datetime(2024, 9, 14)
date_range = pd.date_range(start=start_date, end=end_date)[::-1]  # 역순으로 변환

# 결과를 저장할 리스트
results = []

# 아티스트에 대해 URL을 만들고 데이터 수집
for single_date in date_range:
    if single_date.year == 2024:
        date_str = single_date.strftime('%m.%d')  # 2024년 날짜는 MM.DD 형식
    else:
        date_str = single_date.strftime('%y.%m.%d')  # 2023년 날짜는 YY.MM.DD 형식
    
    total_views = 0
    total_recommendations = 0
    page = 1  # 페이지 초기화

    while True:  # 페이지를 반복하며 수집
        url = f'https://gall.dcinside.com/mgallery/board/lists/?id={first_artist}&page={page}'
        
        # 웹 페이지 요청
        response = requests.get(url)
        
        # 응답이 성공적일 경우
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            posts = soup.find_all('tr', class_='ub-content us-post')  # 게시글 목록
            
            # 게시글이 없다면 종료
            if not posts:
                break
            
            # 해당 날짜의 게시글만 찾기 위한 플래그
            found_posts_for_date = False
            
            for post in posts:
                # 작성일 추출
                post_date = post.find('td', class_='gall_date').text.strip()

                if (single_date.year == 2024 and post_date == date_str) or \
                   (single_date.year == 2023 and post_date == date_str):
                    found_posts_for_date = True
                    
                    # 조회수 및 추천수 추출
                    views = post.find('td', class_='gall_count').text.strip()
                    recommendations = post.find('td', class_='gall_recommend').text.strip()

                    # 조회수와 추천수가 숫자인지 확인 후 합계 계산
                    if views.isdigit():
                        total_views += int(views)
                    if recommendations.isdigit():
                        total_recommendations += int(recommendations)

            # 날짜가 같은 게시글이 발견되었으면 계속 진행
            if found_posts_for_date:
                # 다음 페이지로 이동
                page += 1
            else:
                break  # 같은 날짜의 게시글이 없으면 종료
        else:
            print(f"Error fetching data for {first_artist} on {date_str}: {response.status_code}")
            break
    
    # 결과 리스트에 추가
    results.append({
        '작성일': date_str,
        '총조회수': total_views,
        '총추천수': total_recommendations
    })

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과를 CSV 파일로 저장
results_df.to_csv('output.csv', index=False)
